In [181]:
import causal_nf.utils.wandb_local as wandb_local
import causal_nf.config as causal_nf_config
from causal_nf.config import cfg
import causal_nf.utils.training as causal_nf_train
import os
import glob
import argparse
from yacs.config import CfgNode 
import torch
import causal_nf.utils.io as causal_nf_io
import time
import numpy as np
from causal_nf.preparators.FF_preparator import FFPreparator
from causal_nf.config import cfg
from tqdm import tqdm

In [190]:
ate_folders = ['output_FF_ate1', 'output_FF_ate5'] #output_FF_ate5
models = ['model_1','model_2', 'model_2_binary']#'model_2', 'model_3'
versions = ['v_0','v_1','v_2','v_3','v_4',]#'v_0','v_1','v_2','v_3','v_4',
all_ates = {'output_FF_ate1': {'model_1':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2_binary':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},},
            'output_FF_ate5': {'model_1':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2_binary':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},} }#'output_FF_ate5':{'model_1':[],'model_2':[],'model_3':[],}, 

In [191]:
for ate_folder in ate_folders:
    for model in models:
        for version in versions:
            
            base_path = f'./{ate_folder}/{model}/{version}/'

            try:
                directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
            except:    
                print(f"No directories found in {base_path}. Skip. ")
                continue
            
            # Assuming there's only one directory in A/B/C
            if len(directories)==1:
                file_dir = os.path.join(base_path, directories[0])
                print(file_dir)
            elif len(directories)>1:
                print(f"Multiple directories found: {directories} in base path {base_path}")
            
            
            # file_dir = './output_FF_ate1/model_1/v_0/2e684ef6533a11ef991cacde48001122'
            ckpt_file= f"{file_dir}/last.ckpt"
            
            args_list = []
            args =  CfgNode({'config_file': f'{file_dir}/wandb_local/config_local.yaml', 
                             'config_default_file': f'{file_dir}/wandb_local/default_config.yaml', 
                             'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                             'load_model': f'{file_dir}', 'delete_ckpt': False})
            config = causal_nf_config.build_config(
                config_file=args.config_file,
                args_list=args_list,
                config_default_file=args.config_default_file,
            )
            causal_nf_config.assert_cfg_and_config(cfg, config)
            
            preparator = FFPreparator.loader(cfg.dataset)
            preparator.prepare_data()
            
            # IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
            model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)
            _ = model_lightning.model.eval()
            
            index = 4
            a = 1
            b = 0
            rounds = 5
            
            
            ates = []
            for seed in tqdm(range(rounds)):
                torch.random.manual_seed(seed + 1)
                ate = model_lightning.model.compute_ate(
                    index,
                    a=a,
                    b=b,
                    num_samples=10000,
                    scaler=preparator.scaler_transform,
                )
                ates.append(ate[-1])
            ates = np.array(ates)
            all_ates[ate_folder][model][version] = ates
            
            ate_mean = ates.mean()
            ate_stddev = ates.std()
            print(f"Folder {ate_folder} {model} {version}: Mean ATE over {rounds} rounds = ", ate_mean)
            print(f"Folder {ate_folder} {model} {version}: Stddev ATE over {rounds} rounds = ", ate_stddev)
            

./output_FF_ate1/model_1/v_0/2e684ef6533a11ef991cacde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate1/model_1/v_0/2e684ef6533a11ef991cacde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([2.7091, 2.7169, 2.7340, 2.7486, 0.9532, 1.9621]), scale=te

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.67it/s]


Folder output_FF_ate1 model_1 v_0: Mean ATE over 5 rounds =  0.73728263
Folder output_FF_ate1 model_1 v_0: Stddev ATE over 5 rounds =  0.009643083
./output_FF_ate1/model_1/v_1/4df7458e534311ef807cacde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate1/model_1/v_1/4df

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.73it/s]


Folder output_FF_ate1 model_1 v_1: Mean ATE over 5 rounds =  0.684651
Folder output_FF_ate1 model_1 v_1: Stddev ATE over 5 rounds =  0.009066355
./output_FF_ate1/model_1/v_2/61aead90534511efa976acde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate1/model_1/v_2/61aea

...

Folder output_FF_ate5 model_2 v_2: Mean ATE over 5 rounds =  5.176228
Folder output_FF_ate5 model_2 v_2: Stddev ATE over 5 rounds =  0.008800622
./output_FF_ate5/model_2/v_3/a7c3220253e011efac8facde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate5/model_2/v_3/a7c32

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.81it/s]


Folder output_FF_ate5 model_2 v_3: Mean ATE over 5 rounds =  5.3366995
Folder output_FF_ate5 model_2 v_3: Stddev ATE over 5 rounds =  0.009885517
./output_FF_ate5/model_2/v_4/cac98e6653e111efa5d9acde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate5/model_2/v_4/cac9

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.77it/s]

Folder output_FF_ate5 model_2 v_4: Mean ATE over 5 rounds =  5.010135
Folder output_FF_ate5 model_2 v_4: Stddev ATE over 5 rounds =  0.010306808
No directories found in ./output_FF_ate5/model_2_binary/v_0/. Skip. 
No directories found in ./output_FF_ate5/model_2_binary/v_1/. Skip. 
No directories found in ./output_FF_ate5/model_2_binary/v_2/. Skip. 
No directories found in ./output_FF_ate5/model_2_binary/v_3/. Skip. 
No directories found in ./output_FF_ate5/model_2_binary/v_4/. Skip. 


In [192]:
all_ates

{'output_FF_ate1': {'model_1': {'v_0': array([0.7417052 , 0.7240397 , 0.74745023, 0.7457118 , 0.72750616],
         dtype=float32),
   'v_1': array([0.687008  , 0.67304075, 0.6920202 , 0.69595075, 0.6752355 ],
         dtype=float32),
   'v_2': array([0.7575021 , 0.7394508 , 0.7609292 , 0.76282597, 0.7438395 ],
         dtype=float32),
   'v_3': array([0.75292134, 0.74003637, 0.7587135 , 0.7633687 , 0.74514115],
         dtype=float32),
   'v_4': array([0.73833287, 0.7199769 , 0.744738  , 0.7463745 , 0.7289312 ],
         dtype=float32)},
  'model_2': {'v_0': array([1.0217237, 1.001218 , 1.0315216, 1.0173025, 1.0235791],
         dtype=float32),
   'v_1': array([0.9532697 , 0.93301153, 0.9601879 , 0.94744754, 0.95764005],
         dtype=float32),
   'v_2': array([1.1312631, 1.1124315, 1.1410182, 1.1260133, 1.1326361],
         dtype=float32),
   'v_3': array([1.0754958, 1.0533203, 1.0844492, 1.0696572, 1.0754509],
         dtype=float32),
   'v_4': array([0.8867687, 0.8659631, 0.897896

In [188]:
ate_folder = ate_folders[0]
model = models[2]
version = 'v_0'

base_path = f'./{ate_folder}/{model}/{version}/'

directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

# Assuming there's only one directory in A/B/C
if len(directories)==1:
    file_dir = os.path.join(base_path, directories[0])
    print(file_dir)
elif len(directories)>1:
    print(f"Multiple directories found: {directories} in base path {base_path}")
else:
    print(f"No directories found in {base_path}. Skip. ")


# file_dir = './output_FF_ate1/model_1/v_0/2e684ef6533a11ef991cacde48001122'
ckpt_file= f"{file_dir}/last.ckpt"

args_list = []
args =  CfgNode({'config_file': f'{file_dir}/wandb_local/config_local.yaml', 
                 'config_default_file': f'{file_dir}/wandb_local/default_config.yaml', 
                 'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                 'load_model': f'{file_dir}', 'delete_ckpt': False})
config = causal_nf_config.build_config(
    config_file=args.config_file,
    args_list=args_list,
    config_default_file=args.config_default_file,
)
causal_nf_config.assert_cfg_and_config(cfg, config)

preparator = FFPreparator.loader(cfg.dataset)
preparator.prepare_data()

# IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)
_ = model_lightning.model.eval()

index = 4
a = 1
b = 0
rounds = 5


ates = []
for seed in tqdm(range(rounds)):
    torch.random.manual_seed(seed + 1)
    ate = model_lightning.model.compute_ate(
        index,
        a=a,
        b=b,
        num_samples=10000,
        scaler=preparator.scaler_transform,
    )
    ates.append(ate[-1])
ates = np.array(ates)
print(ates)
ate_mean = ates.mean()
ate_stddev = ates.std()
print(f"Folder {ate_folder} {model} {version}: Mean ATE over {rounds} rounds = ", ate_mean)
print(f"Folder {ate_folder} {model} {version}: Stddev ATE over {rounds} rounds = ", ate_stddev)

./output_FF_ate1/model_2_binary/v_0/5d4482e853fe11efbdb8acde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate1/model_2_binary/v_0/5d4482e853fe11efbdb8acde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([7.4150, 7.4221, 0.5011, 0.4884, 0.9722, 1.98

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.53it/s]

[1.0217237 1.001218  1.0315216 1.0173025 1.0235791]
Folder output_FF_ate1 model_2_binary v_0: Mean ATE over 5 rounds =  1.019069
Folder output_FF_ate1 model_2_binary v_0: Stddev ATE over 5 rounds =  0.010042261


In [178]:
all_ates_stats = {'output_FF_ate1': {'model_1':{'mean':[], 'std':[]}, 'model_2':{'mean':[], 'std':[]}, },
                 'output_FF_ate5': {'model_1':{'mean':[], 'std':[]}, 'model_2':{'mean':[], 'std':[]}, } }#'output_FF_ate5':{'model_1':[],'model_2':[],'model_3':[],}, 

In [179]:
for ate_folder in ate_folders:
    for model in models:
        ates_model = []
        for version in versions:
            ates_v = all_ates[ate_folder][model][version]
            ates_model += list(ates_v)
        print(ates_model)
        all_ates_stats[ate_folder][model]['mean'] = np.array(ates_model).mean()
        all_ates_stats[ate_folder][model]['std'] = np.array(ates_model).std()
        
        

[0.7417052, 0.7240397, 0.74745023, 0.7457118, 0.72750616, 0.687008, 0.67304075, 0.6920202, 0.69595075, 0.6752355, 0.7575021, 0.7394508, 0.7609292, 0.76282597, 0.7438395, 0.75292134, 0.74003637, 0.7587135, 0.7633687, 0.74514115, 0.73833287, 0.7199769, 0.744738, 0.7463745, 0.7289312]
[1.0217237, 1.001218, 1.0315216, 1.0173025, 1.0235791, 0.9532697, 0.93301153, 0.9601879, 0.94744754, 0.95764005, 1.1312631, 1.1124315, 1.1410182, 1.1260133, 1.1326361, 1.0754958, 1.0533203, 1.0844492, 1.0696572, 1.0754509, 0.8867687, 0.8659631, 0.8978963, 0.8817812, 0.8925307]
[4.0958853, 4.0691714, 4.093867, 4.0952916, 4.0677905, 4.1992354, 4.1772623, 4.1998177, 4.205276, 4.180587, 4.397782, 4.37084, 4.3951674, 4.402388, 4.377641, 4.228791, 4.2055836, 4.2282324, 4.237474, 4.2113543, 4.247327, 4.220379, 4.246558, 4.252003, 4.233597]
[4.6968203, 4.676272, 4.7078085, 4.6925735, 4.701911, 4.9422026, 4.9242163, 4.948625, 4.9395156, 4.9482746, 5.1804495, 5.160677, 5.187171, 5.1744494, 5.178393, 5.342574, 5.319396

In [180]:
all_ates_stats

{'output_FF_ate1': {'model_1': {'mean': 0.7325101, 'std': 0.026673816},
  'model_2': {'mean': 1.010943, 'std': 0.08679322}},
 'output_FF_ate5': {'model_1': {'mean': 4.225572, 'std': 0.09858912},
  'model_2': {'mean': 5.031741, 'std': 0.21747321}}}